In [2]:
from secdiagai.classifier.base import WebClassifier
from secdiagai.classifier.classifier_maker import create_web_classifier
from secdiagai.classifier.clf_rules import rule_login
from secdiagai.common.classification_util import make_scorer_with_rule
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from scripts.tools.cv.cv_util import load_and_create_X_y_from_json
from sklearn.metrics import accuracy_score
from scripts.utils.debug_util import print_attrs

JSONS_BASE = '/Users/hirokisugiyama/Work/NTTTX/NTTTX_201709_/data/labels/2017-09-27_v5_part_10'
# JSONS_BASE = '/Users/hirokisugiyama/Work/NTTTX/NTTTX_201709_/data/labels/2017-09-27_v5'
type_label = 'login'
X, y = load_and_create_X_y_from_json(JSONS_BASE, type_label)



In [30]:
RandomForestClassifier().get_params()

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [24]:
# def scorer(estimator, X, y, sample_weight=None):
#     y_pred = estimator.predict(X)

#     # if rule is not None:
#     #     # ルールが指定されている場合は適用する
#     #     y = [rule(x, y[i]) for i, x in enumerate(X)]
#     y_pred = [rule_login(x, y_pred[i]) for i, x in enumerate(X)]

#     return accuracy_score(y, y_pred, sample_weight=sample_weight)


def try_web_clf():
    type_label = 'login'
    core_clf = RandomForestClassifier()
    w_clf = create_web_classifier(type_label, core_clf)


core_clf = RandomForestClassifier()
w_clf = create_web_classifier(type_label, core_clf)

param_grid = [
    {
    'clf__n_estimators': [5, 10],
    'clf__max_features': [None],
    'clf__random_state': [42]
},
    {
    'clf__n_estimators': [5],
    'clf__max_features': [None, 'sqrt'],
    'clf__random_state': [42]
},
    
]

# grid_with_scorer = GridSearchCV(w_clf._clf, param_grid, scoring=scorer)
grid_without_scorer = GridSearchCV(w_clf._clf, param_grid)
grid_with_scorer = GridSearchCV(w_clf._clf, param_grid, scoring=make_scorer_with_rule(rule_login))


In [25]:
grid_without_scorer.fit(X, y)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('feature_extraction', FeatureUnion(n_jobs=1,
       transformer_list=[('button', Pipeline(memory=None,
     steps=[('extract', <secdiagai.classifier.transformers.FormSubmitButtonNameTransformer object at 0x10f576a58>), ('count', CountVectorizer(analyzer=<function create_count_vectorizer.<loc...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'clf__n_estimators': [5, 10], 'clf__max_features': [None], 'clf__random_state': [42]}, {'clf__n_estimators': [5], 'clf__max_features': [None, 'sqrt'], 'clf__random_state': [42]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [26]:
grid_with_scorer.fit(X, y)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('feature_extraction', FeatureUnion(n_jobs=1,
       transformer_list=[('button', Pipeline(memory=None,
     steps=[('extract', <secdiagai.classifier.transformers.FormSubmitButtonNameTransformer object at 0x10f576a58>), ('count', CountVectorizer(analyzer=<function create_count_vectorizer.<loc...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'clf__n_estimators': [5, 10], 'clf__max_features': [None], 'clf__random_state': [42]}, {'clf__n_estimators': [5], 'clf__max_features': [None, 'sqrt'], 'clf__random_state': [42]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=functools.partial(<function make_scorer_with_rule.<locals>.scorer at 0x11802aea0>, rule=<function rule_login at 0x10f78ee18>),
       verbose=0)

In [27]:
print(grid_without_scorer.best_score_)
print(grid_without_scorer.best_params_)

0.939814814815
{'clf__max_features': None, 'clf__n_estimators': 5, 'clf__random_state': 42}


In [28]:
print(grid_with_scorer.best_score_)
print(grid_with_scorer.best_params_)
print(grid_with_scorer.best_index_)

0.981481481481
{'clf__max_features': None, 'clf__n_estimators': 5, 'clf__random_state': 42}
0


In [29]:
for k, v in grid_with_scorer.cv_results_.items():
    print(k)
    print(v)
    print('')

mean_fit_time
[ 1.67594163  1.84189741  1.70422014  1.55429626]

std_fit_time
[ 0.37319183  0.81779101  0.54571349  0.53744005]

mean_score_time
[ 0.94201819  1.24091029  1.04816898  0.86178374]

std_score_time
[ 0.57595052  0.56791928  0.56947787  0.38310158]

param_clf__max_features
[None None None 'sqrt']

param_clf__n_estimators
[5 10 5 5]

param_clf__random_state
[42 42 42 42]

params
[{'clf__max_features': None, 'clf__n_estimators': 5, 'clf__random_state': 42}, {'clf__max_features': None, 'clf__n_estimators': 10, 'clf__random_state': 42}, {'clf__max_features': None, 'clf__n_estimators': 5, 'clf__random_state': 42}, {'clf__max_features': 'sqrt', 'clf__n_estimators': 5, 'clf__random_state': 42}]

split0_test_score
[ 0.97260274  0.97260274  0.97260274  0.95890411]

split1_test_score
[ 0.98611111  0.98611111  0.98611111  0.98611111]

split2_test_score
[ 0.98591549  0.98591549  0.98591549  0.97183099]

mean_test_score
[ 0.98148148  0.98148148  0.98148148  0.97222222]

std_test_score
[